In [4]:
import mysql.connector
from datetime import datetime
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox, simpledialog

def conectar():
    """Establece la conexión con la base de datos MySQL."""
    try:
        conn = mysql.connector.connect(
            host="shortline.proxy.rlwy.net",
            port=47316,
            user="root",
            password="cHHzHOJVpxFjPXLJsUQPHMNiZhCECXfJ",
            database="gamestore_db"
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error de conexión a la base de datos (desde conectar()): {err}")
        return None

In [5]:
class GameStoreManager:
    def __init__(self):
        self.db_conn = conectar()
        if self.db_conn:
            print("GameStoreManager: Conectado a la base de datos.")
            self._crear_tablas_si_no_existen()
        else:
            print("GameStoreManager: No se pudo conectar a la base de datos.")

    def _execute_query(self, query, params=None, fetch=None, commit=False, is_ddl=False):
        if not self.db_conn or not self.db_conn.is_connected():
            print("GameStoreManager: Intentando reconectar...")
            self.db_conn = conectar()
            if not self.db_conn:
                print("GameStoreManager: Fallo al reconectar.")
                return None if fetch else False

        use_dictionary_cursor = isinstance(fetch, str) and 'dict' in fetch
        cursor = None
        try:
            cursor = self.db_conn.cursor(dictionary=use_dictionary_cursor)
            cursor.execute(query, params or ())
            if commit or is_ddl:
                self.db_conn.commit()
            if fetch:
                if fetch == 'one' or fetch == 'dict_one':
                    return cursor.fetchone()
                elif fetch == 'all' or fetch == 'dict_all':
                    return cursor.fetchall()
            return True
        except mysql.connector.Error as err:
            print(f"GameStoreManager DB Error: {err} (Query: {query[:100]}... Params: {params})")
            if self.db_conn and self.db_conn.is_connected() and commit:
                try: self.db_conn.rollback()
                except mysql.connector.Error as roll_err: print(f"Rollback Error: {roll_err}")
            return None if fetch else False
        finally:
            if cursor: cursor.close()

    def _crear_tablas_si_no_existen(self):
        queries = {
            "videojuegos": """
                CREATE TABLE IF NOT EXISTS videojuegos (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    nombre VARCHAR(255) NOT NULL,
                    categoria VARCHAR(100),
                    precio DECIMAL(10, 2) NOT NULL,
                    consola VARCHAR(100),
                    genero VARCHAR(100),
                    cantidad INT DEFAULT 0
                );""",
            "clientes": """
                CREATE TABLE IF NOT EXISTS clientes (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    nombre VARCHAR(255) NOT NULL,
                    email VARCHAR(255) UNIQUE
                );""",
            "ventas": """
                CREATE TABLE IF NOT EXISTS ventas (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    id_cliente INT,
                    id_videojuego INT,
                    nombre_videojuego_vendido VARCHAR(255) NOT NULL,
                    precio_venta DECIMAL(10, 2) NOT NULL, -- Precio unitario al momento de la venta
                    cantidad_vendida INT DEFAULT 1,
                    fecha_venta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (id_cliente) REFERENCES clientes(id) ON DELETE SET NULL,
                    FOREIGN KEY (id_videojuego) REFERENCES videojuegos(id) ON DELETE RESTRICT
                );"""
        }
        for tabla, query in queries.items():
            if self._execute_query(query, is_ddl=True):
                print(f"GameStoreManager: Tabla '{tabla}' verificada/creada.")
            else:
                print(f"GameStoreManager: Error al crear/verificar tabla '{tabla}'.")

    def is_connected(self):
        return self.db_conn and self.db_conn.is_connected()

    def add_videojuego_gui(self, nombre, categoria, precio, consola, genero, cantidad):
        if not nombre or precio is None or cantidad is None:
            return False, "Nombre, Precio y Cantidad son obligatorios."
        try:
            precio_float = float(precio)
            cantidad_int = int(cantidad)
            if precio_float < 0:
                return False, "El precio no puede ser negativo."
            if cantidad_int < 0:
                return False, "La cantidad no puede ser negativa."
        except ValueError:
            return False, "Precio y Cantidad deben ser números válidos."

        query = "INSERT INTO videojuegos (nombre, categoria, precio, consola, genero, cantidad) VALUES (%s, %s, %s, %s, %s, %s)"
        params = (nombre, categoria, precio_float, consola, genero, cantidad_int)
        if self._execute_query(query, params, commit=True):
            return True, f"Videojuego '{nombre}' agregado exitosamente con stock {cantidad_int}."
        else:
            return False, f"Error al agregar videojuego '{nombre}'."

    # NUEVO: Método para actualizar stock de un videojuego
    def update_videojuego_stock_gui(self, juego_id, nueva_cantidad_total):
        if juego_id is None:
            return False, "No se seleccionó ningún videojuego."
        try:
            cantidad_int = int(nueva_cantidad_total)
            if cantidad_int < 0:
                return False, "La nueva cantidad total no puede ser negativa."
        except ValueError:
            return False, "La nueva cantidad total debe ser un número entero válido."

        query = "UPDATE videojuegos SET cantidad = %s WHERE id = %s"
        params = (cantidad_int, juego_id)
        if self._execute_query(query, params, commit=True):
            return True, f"Stock del videojuego ID {juego_id} actualizado a {cantidad_int} unidades."
        else:
            return False, f"Error al actualizar el stock del videojuego ID {juego_id}."

    def add_cliente_gui(self, nombre, email):
        if not nombre:
            return False, "El nombre del cliente es obligatorio."
        email_param = email if email else None
        query = "INSERT INTO clientes (nombre, email) VALUES (%s, %s)"
        params = (nombre, email_param)
        if self._execute_query(query, params, commit=True):
            return True, f"Cliente '{nombre}' agregado exitosamente."
        else:
            return False, f"Error al agregar cliente '{nombre}'. Verifique si el email ya existe."

    def delete_cliente_gui(self, cliente_id):
        if cliente_id is None:
            return False, "No se seleccionó ningún cliente."
        query = "DELETE FROM clientes WHERE id = %s"
        if self._execute_query(query, (cliente_id,), commit=True):
            return True, f"Cliente ID {cliente_id} borrado exitosamente."
        else:
            return False, f"Error al borrar el cliente ID {cliente_id}."

    def get_catalogo_videojuegos(self, solo_con_stock=False): # Añadido parámetro opcional
        query = "SELECT id, nombre, categoria, precio, consola, genero, cantidad FROM videojuegos"
        if solo_con_stock:
            query += " WHERE cantidad > 0"
        query += " ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')

    def get_clientes_for_selection(self):
        query = "SELECT id, nombre, email FROM clientes ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')

    def get_videojuego_by_id(self, juego_id):
        query = "SELECT id, nombre, precio, cantidad FROM videojuegos WHERE id = %s"
        return self._execute_query(query, (juego_id,), fetch='dict_one')

    # MODIFICADO: Lógica de venta para manejar la cantidad_a_vender
    def realizar_venta_gui(self, id_videojuego, id_cliente, cantidad_a_vender):
        if not isinstance(cantidad_a_vender, int) or cantidad_a_vender <= 0:
            return False, "La cantidad a vender debe ser un número entero positivo."

        juego_info = self.get_videojuego_by_id(id_videojuego)
        if not juego_info:
            return False, f"Error: Videojuego con ID {id_videojuego} no encontrado."

        nombre_videojuego_vendido = juego_info['nombre']
        precio_venta_unitario = juego_info['precio']
        stock_actual = juego_info['cantidad']

        if stock_actual < cantidad_a_vender:
            return False, f"Stock insuficiente para '{nombre_videojuego_vendido}'. Disponible: {stock_actual}, Solicitado: {cantidad_a_vender}."

        cliente_id_param = id_cliente if id_cliente is not None else None
        query_venta = "INSERT INTO ventas (id_cliente, id_videojuego, nombre_videojuego_vendido, precio_venta, cantidad_vendida, fecha_venta) VALUES (%s, %s, %s, %s, %s, %s)"
        fecha_actual = datetime.now()
        params_venta = (cliente_id_param, id_videojuego, nombre_videojuego_vendido, precio_venta_unitario, cantidad_a_vender, fecha_actual)

        if not self._execute_query(query_venta, params_venta, commit=True):
            return False, "Error al registrar la venta en la base de datos."

        nuevo_stock = stock_actual - cantidad_a_vender
        query_actualizar_stock = "UPDATE videojuegos SET cantidad = %s WHERE id = %s"
        params_actualizar_stock = (nuevo_stock, id_videojuego)

        if self._execute_query(query_actualizar_stock, params_actualizar_stock, commit=True):
            return True, f"Venta de '{nombre_videojuego_vendido}' (x{cantidad_a_vender}) completada. Stock actualizado a {nuevo_stock}."
        else:
            return False, (f"Venta de '{nombre_videojuego_vendido}' (x{cantidad_a_vender}) registrada, "
                           f"pero ¡ERROR al actualizar el stock! Por favor, verifique manualmente. ID Juego: {id_videojuego}")


    def get_all_sales_report(self):
        query = """
            SELECT v.id, c.nombre AS nombre_cliente, v.nombre_videojuego_vendido,
                   v.precio_venta, v.cantidad_vendida, v.fecha_venta
            FROM ventas v
            LEFT JOIN clientes c ON v.id_cliente = c.id
            ORDER BY v.fecha_venta DESC;
        """
        return self._execute_query(query, fetch='dict_all')

    def get_total_revenue_report(self):
        query = "SELECT SUM(precio_venta * cantidad_vendida) AS total FROM ventas"
        resultado = self._execute_query(query, fetch='dict_one')
        if resultado and resultado['total'] is not None:
            return resultado['total'], "Ingresos totales calculados."
        elif resultado and resultado['total'] is None:
            return 0.0, "No hay ventas registradas."
        else:
            return None, "Error al calcular ingresos totales."

    def get_sales_by_client_report(self):
        query = """
            SELECT c.nombre AS nombre_cliente, COUNT(v.id) AS numero_transacciones,
                   SUM(v.cantidad_vendida) AS total_juegos_comprados,
                   SUM(v.precio_venta * v.cantidad_vendida) AS total_gastado
            FROM clientes c
            JOIN ventas v ON c.id = v.id_cliente
            GROUP BY c.id, c.nombre
            ORDER BY total_gastado DESC;
        """
        report_data = self._execute_query(query, fetch='dict_all')
        if report_data is None:
            return None, "Error al generar el informe de ventas por cliente."
        if not report_data:
            return [], "No hay ventas registradas o clientes con ventas."
        return report_data, "Informe de ventas por cliente generado."

    def cerrar_conexion(self):
        if self.db_conn and self.db_conn.is_connected():
            self.db_conn.close()
            print("GameStoreManager: Conexión cerrada.")

In [6]:
class GameStoreApp:
    def __init__(self, root_window):
        self.root = root_window
        self.root.title("GameStore Manager")
        self.root.geometry("950x700") # Ajustado para nuevo botón y campos

        self.manager = GameStoreManager()

        if not self.manager.is_connected():
            messagebox.showerror("Error de Conexión",
                                 "No se pudo conectar a la base de datos.\n"
                                 "La aplicación no funcionará correctamente.\n"
                                 "Revise la consola para más detalles.")

        style = ttk.Style()
        style.theme_use('clam') # 'clam', 'alt', 'default', 'classic'

        # Configurar estilos para Treeview y otros widgets si es necesario
        style.configure("Treeview.Heading", font=('Arial', 10, 'bold'))
        style.configure("TButton", padding=6, relief="flat", font=('Arial', 10))
        style.configure("TLabel", font=('Arial', 10))
        style.configure("TEntry", padding=5, font=('Arial', 10))
        style.configure("TCombobox", padding=5, font=('Arial', 10))


        menu_frame = ttk.Frame(self.root, padding="10")
        menu_frame.pack(side=tk.TOP, fill=tk.X)

        ttk.Button(menu_frame, text="Agregar Videojuego", command=self.open_add_videojuego_window).pack(side=tk.LEFT, padx=5, pady=5)
        ttk.Button(menu_frame, text="Modificar Stock", command=self.open_update_stock_window).pack(side=tk.LEFT, padx=5, pady=5) # NUEVO BOTÓN
        ttk.Button(menu_frame, text="Agregar Cliente", command=self.open_add_cliente_window).pack(side=tk.LEFT, padx=5, pady=5)
        ttk.Button(menu_frame, text="Borrar Cliente", command=self.open_delete_cliente_window).pack(side=tk.LEFT, padx=5, pady=5)
        ttk.Button(menu_frame, text="Realizar Venta", command=self.open_realizar_venta_window).pack(side=tk.LEFT, padx=5, pady=5)
        ttk.Button(menu_frame, text="Informes de Ventas", command=self.open_informes_window).pack(side=tk.LEFT, padx=5, pady=5)

        self.content_frame = ttk.Frame(self.root, padding="10")
        self.content_frame.pack(expand=True, fill=tk.BOTH)

        self.show_catalogo_in_content_frame()


    def clear_content_frame(self):
        for widget in self.content_frame.winfo_children():
            widget.destroy()

    def open_add_videojuego_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Agregar Videojuego")
        top.geometry("400x350")

        fields = ["Nombre", "Categoría", "Precio", "Consola", "Género", "Cantidad Inicial"]
        entries = {}

        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry

        form_frame.grid_columnconfigure(1, weight=1)

        def submit_videojuego():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"] or not data["Precio"] or not data["Cantidad Inicial"]:
                messagebox.showerror("Error de Validación", "Nombre, Precio y Cantidad Inicial son obligatorios.", parent=top)
                return
            try:
                float(data["Precio"])
                int(data["Cantidad Inicial"])
            except ValueError:
                messagebox.showerror("Error de Validación", "Precio debe ser un número, Cantidad Inicial debe ser un entero.", parent=top)
                return

            success, message = self.manager.add_videojuego_gui(
                data["Nombre"], data["Categoría"], data["Precio"],
                data["Consola"], data["Género"], data["Cantidad Inicial"]
            )
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame()
            else:
                messagebox.showerror("Error", message, parent=top)

        ttk.Button(form_frame, text="Agregar", command=submit_videojuego).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    # NUEVO: Ventana para modificar stock
    def open_update_stock_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Modificar Stock de Videojuego")
        top.geometry("550x300")

        main_frame = ttk.Frame(top, padding="10")
        main_frame.pack(expand=True, fill=tk.BOTH)

        ttk.Label(main_frame, text="Seleccionar Videojuego:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)

        videojuegos_disponibles = self.manager.get_catalogo_videojuegos() # Obtener todos, no solo con stock
        juego_options = []
        self.update_stock_juego_map = {}
        current_stock_label_var = tk.StringVar(value="Stock actual: N/A")

        if videojuegos_disponibles:
            for j in videojuegos_disponibles:
                display_name = f"ID: {j['id']} - {j['nombre']} (Stock: {j['cantidad']})"
                juego_options.append(display_name)
                self.update_stock_juego_map[display_name] = {'id': j['id'], 'stock_actual': j['cantidad']}
        
        selected_juego_str = tk.StringVar()
        juego_combo = ttk.Combobox(main_frame, textvariable=selected_juego_str,
                                     values=juego_options, width=60, state="readonly")
        juego_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)

        ttk.Label(main_frame, textvariable=current_stock_label_var).grid(row=1, column=1, padx=5, pady=2, sticky=tk.W)

        def on_juego_selected(event):
            selected_display_name = selected_juego_str.get()
            juego_info = self.update_stock_juego_map.get(selected_display_name)
            if juego_info:
                current_stock_label_var.set(f"Stock actual: {juego_info['stock_actual']}")
            else:
                current_stock_label_var.set("Stock actual: N/A")

        juego_combo.bind("<<ComboboxSelected>>", on_juego_selected)

        if juego_options:
            juego_combo.current(0)
            on_juego_selected(None) # Para inicializar el stock actual
        else:
            juego_combo.set("No hay videojuegos registrados.")
            juego_combo.config(state=tk.DISABLED)


        ttk.Label(main_frame, text="Nueva Cantidad Total:").grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
        nueva_cantidad_entry = ttk.Entry(main_frame, width=15)
        nueva_cantidad_entry.grid(row=2, column=1, padx=5, pady=5, sticky=tk.W) # Alineado a la izquierda

        main_frame.grid_columnconfigure(1, weight=1) # Para que el combobox se expanda

        def submit_update_stock():
            juego_display_name = selected_juego_str.get()
            if not juego_display_name or juego_display_name == "No hay videojuegos registrados.":
                messagebox.showerror("Error", "Debe seleccionar un videojuego.", parent=top)
                return

            juego_info = self.update_stock_juego_map.get(juego_display_name)
            if not juego_info:
                messagebox.showerror("Error", "Selección de videojuego inválida.", parent=top)
                return

            nueva_cantidad_str = nueva_cantidad_entry.get().strip()
            if not nueva_cantidad_str:
                messagebox.showerror("Error de Validación", "Debe ingresar la nueva cantidad total.", parent=top)
                return
            
            try:
                nueva_cantidad_int = int(nueva_cantidad_str)
                if nueva_cantidad_int < 0:
                    messagebox.showerror("Error de Validación", "La cantidad no puede ser negativa.", parent=top)
                    return
            except ValueError:
                messagebox.showerror("Error de Validación", "La nueva cantidad total debe ser un número entero válido.", parent=top)
                return

            success, message = self.manager.update_videojuego_stock_gui(juego_info['id'], nueva_cantidad_int)
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame() # Actualizar vista del catálogo
            else:
                messagebox.showerror("Error", message, parent=top)

        update_button = ttk.Button(main_frame, text="Actualizar Stock", command=submit_update_stock)
        update_button.grid(row=3, column=0, columnspan=2, pady=15)
        if not juego_options:
            update_button.config(state=tk.DISABLED)

        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def open_add_cliente_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Agregar Cliente")
        top.geometry("350x200")

        fields = ["Nombre", "Email"]
        entries = {}
        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry

        form_frame.grid_columnconfigure(1, weight=1)

        def submit_cliente():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"]:
                messagebox.showerror("Error de Validación", "El nombre es obligatorio.", parent=top)
                return

            success, message = self.manager.add_cliente_gui(data["Nombre"], data["Email"])
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
            else:
                messagebox.showerror("Error", message, parent=top)

        ttk.Button(form_frame, text="Agregar", command=submit_cliente).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    def open_delete_cliente_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Borrar Cliente")
        top.geometry("450x250") # Un poco más de espacio

        main_frame = ttk.Frame(top, padding="10")
        main_frame.pack(expand=True, fill=tk.BOTH)

        ttk.Label(main_frame, text="Seleccionar Cliente a Borrar:").pack(pady=(0,5), anchor=tk.W)

        clientes_disponibles = self.manager.get_clientes_for_selection()
        cliente_options = []
        self.delete_cliente_map = {}
        if clientes_disponibles:
            for c in clientes_disponibles:
                display_name = f"{c['nombre']} (ID: {c['id']})"
                if c.get('email'):
                    display_name += f" - Email: {c['email']}"
                cliente_options.append(display_name)
                self.delete_cliente_map[display_name] = c['id']
        
        selected_cliente_to_delete_str = tk.StringVar()
        cliente_combo = ttk.Combobox(main_frame, textvariable=selected_cliente_to_delete_str,
                                     values=cliente_options, width=50, state="readonly")
        cliente_combo.pack(fill=tk.X, pady=5)

        delete_button = ttk.Button(main_frame, text="Borrar Cliente Seleccionado", command=lambda: submit_delete_cliente(selected_cliente_to_delete_str.get(), top))
        delete_button.pack(pady=10)

        if cliente_options:
            cliente_combo.current(0)
        else:
            ttk.Label(main_frame, text="No hay clientes para borrar.").pack(pady=10)
            cliente_combo.config(state=tk.DISABLED)
            delete_button.config(state=tk.DISABLED)


        def submit_delete_cliente(cliente_display_name, top_level_window):
            if not cliente_display_name:
                messagebox.showerror("Error", "Debe seleccionar un cliente.", parent=top_level_window)
                return

            cliente_id_a_borrar = self.delete_cliente_map.get(cliente_display_name)
            if cliente_id_a_borrar is None:
                messagebox.showerror("Error", "Selección de cliente inválida.", parent=top_level_window)
                return

            confirm = messagebox.askyesno("Confirmar Borrado",
                                          f"¿Está seguro que desea borrar al cliente '{cliente_display_name}'?\n"
                                          "Esto no borrará sus ventas, pero las desvinculará de este cliente.",
                                          parent=top_level_window)
            if not confirm:
                return

            success, message = self.manager.delete_cliente_gui(cliente_id_a_borrar)
            if success:
                messagebox.showinfo("Éxito", message, parent=top_level_window)
                top_level_window.destroy()
            else:
                messagebox.showerror("Error", message, parent=top_level_window)
        
        ttk.Button(main_frame, text="Cancelar", command=top.destroy).pack(pady=5)
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def show_catalogo_in_content_frame(self, event=None):
        self.clear_content_frame()
        if not self.manager.is_connected():
            ttk.Label(self.content_frame, text="No hay conexión a la base de datos para mostrar el catálogo.", foreground="red").pack(pady=20)
            return

        catalogo = self.manager.get_catalogo_videojuegos()

        if catalogo is None:
            ttk.Label(self.content_frame, text="Error al cargar el catálogo.", foreground="red").pack(pady=20)
            return
        if not catalogo:
            ttk.Label(self.content_frame, text="El catálogo de videojuegos está vacío.").pack(pady=20)
            return

        cols = ('ID', 'Nombre', 'Categoría', 'Precio', 'Consola', 'Género', 'Stock')
        tree = ttk.Treeview(self.content_frame, columns=cols, show='headings', selectmode="browse")

        for col_name in cols:
            tree.heading(col_name, text=col_name)
            tree.column(col_name, width=110, anchor=tk.W, minwidth=60) # Ajuste general
        
        tree.column('ID', width=50, anchor=tk.CENTER, stretch=tk.NO)
        tree.column('Nombre', width=220)
        tree.column('Precio', anchor=tk.E, width=80)
        tree.column('Stock', anchor=tk.CENTER, width=70)


        for juego in catalogo:
            tree.insert("", tk.END, values=(
                juego['id'], juego['nombre'], juego.get('categoria', ''),
                f"${juego['precio']:.2f}",
                juego.get('consola', ''), juego.get('genero', ''),
                juego.get('cantidad', 0)
            ))

        vsb = ttk.Scrollbar(self.content_frame, orient="vertical", command=tree.yview)
        hsb = ttk.Scrollbar(self.content_frame, orient="horizontal", command=tree.xview)
        tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

        tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        hsb.pack(side=tk.BOTTOM, fill=tk.X)


    def open_realizar_venta_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Realizar Venta")
        top.geometry("650x400") # Ajustado para el campo de cantidad

        selection_frame = ttk.Frame(top, padding="10")
        selection_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # Frame para cantidad
        cantidad_frame = ttk.Frame(top, padding="10")
        cantidad_frame.pack(fill=tk.X, padx=10, pady=5)

        action_frame = ttk.Frame(top, padding="10")
        action_frame.pack(fill=tk.X, padx=10, pady=10)

        # --- Selección de Videojuego ---
        ttk.Label(selection_frame, text="Videojuego:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        juegos_disponibles_data = self.manager.get_catalogo_videojuegos(solo_con_stock=True) # Solo juegos con stock > 0
        juego_options = []
        self.juego_map_venta = {} 
        if juegos_disponibles_data:
            for j in juegos_disponibles_data:
                display_name = f"{j['nombre']} (${j['precio']:.2f}) - Stock: {j['cantidad']}"
                juego_options.append(display_name)
                self.juego_map_venta[display_name] = {'id': j['id'], 'nombre': j['nombre'], 'precio': j['precio'], 'stock_actual': j['cantidad']}
        
        selected_juego_str = tk.StringVar()
        juego_combo = ttk.Combobox(selection_frame, textvariable=selected_juego_str,
                                   values=juego_options, width=60, state="readonly")
        juego_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)
        
        venta_button_state = tk.NORMAL
        if not juego_options:
            juego_combo.set("No hay juegos con stock")
            juego_combo.config(state=tk.DISABLED)
            venta_button_state = tk.DISABLED
        elif juego_options:
            juego_combo.current(0)

        # --- Selección de Cliente ---
        ttk.Label(selection_frame, text="Cliente:").grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        clientes_disponibles = self.manager.get_clientes_for_selection()
        cliente_options = ["[Cliente Anónimo/Invitado]"]
        self.cliente_map_venta = {"[Cliente Anónimo/Invitado]": None}

        if clientes_disponibles:
            for c in clientes_disponibles:
                display_name = f"{c['nombre']} (ID: {c['id']})"
                cliente_options.append(display_name)
                self.cliente_map_venta[display_name] = c['id']

        selected_cliente_str = tk.StringVar()
        cliente_combo = ttk.Combobox(selection_frame, textvariable=selected_cliente_str,
                                     values=cliente_options, width=60, state="readonly")
        cliente_combo.grid(row=1, column=1, padx=5, pady=5, sticky=tk.EW)
        if cliente_options:
            cliente_combo.current(0)
        
        selection_frame.grid_columnconfigure(1, weight=1)

        # --- Cantidad a Vender ---
        ttk.Label(cantidad_frame, text="Cantidad a Vender:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.cantidad_a_vender_var = tk.IntVar(value=1)
        cantidad_spinbox = ttk.Spinbox(cantidad_frame, from_=1, to=99, textvariable=self.cantidad_a_vender_var, width=5, state="readonly") # Max 99, o ajustar
        cantidad_spinbox.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)


        def submit_venta():
            juego_display_name = selected_juego_str.get()
            cliente_display_name = selected_cliente_str.get()
            
            try:
                cantidad_vendida = int(self.cantidad_a_vender_var.get())
                if cantidad_vendida <= 0:
                    messagebox.showerror("Error", "La cantidad a vender debe ser mayor que cero.", parent=top)
                    return
            except ValueError:
                messagebox.showerror("Error", "Cantidad a vender inválida.", parent=top)
                return

            if not juego_display_name or juego_display_name == "No hay juegos con stock":
                messagebox.showerror("Error", "Debe seleccionar un videojuego con stock disponible.", parent=top)
                return
            if not cliente_display_name:
                 messagebox.showerror("Error", "Debe seleccionar un cliente (puede ser Anónimo).", parent=top)
                 return

            juego_seleccionado_info = self.juego_map_venta.get(juego_display_name)
            id_cliente_seleccionado = self.cliente_map_venta.get(cliente_display_name)

            if not juego_seleccionado_info:
                messagebox.showerror("Error", "Selección de videojuego inválida.", parent=top)
                return
            
            # Re-verificar stock antes de confirmar (la info en juego_seleccionado_info es del momento de cargar la ventana)
            juego_actual_db = self.manager.get_videojuego_by_id(juego_seleccionado_info['id'])
            if not juego_actual_db or juego_actual_db['cantidad'] < cantidad_vendida:
                messagebox.showerror("Error de Stock", 
                                     f"No hay suficiente stock para '{juego_seleccionado_info['nombre']}' (Solicitado: {cantidad_vendida}, Disponible: {juego_actual_db['cantidad'] if juego_actual_db else 0}).\n"
                                     "Refrescando opciones de venta...", 
                                     parent=top)
                # Guardar selección actual de cliente si es posible y recargar ventana de venta
                current_cliente_sel = selected_cliente_str.get()
                top.destroy()
                self.open_realizar_venta_window() 
                # Tratar de restaurar la selección del cliente (requeriría pasarla o almacenarla)
                # Por ahora, el usuario tendrá que re-seleccionar cliente si la ventana se recarga.
                return

            precio_total_estimado = juego_actual_db['precio'] * cantidad_vendida
            confirm_msg = (f"Vender {cantidad_vendida} unidad(es) de '{juego_actual_db['nombre']}'\n"
                           f"Precio Unitario: ${juego_actual_db['precio']:.2f}\n"
                           f"Precio Total: ${precio_total_estimado:.2f}\n"
                           f"Al cliente: {cliente_display_name} (ID: {id_cliente_seleccionado if id_cliente_seleccionado else 'Anónimo'})?\n"
                           f"Stock actual: {juego_actual_db['cantidad']} -> Nuevo stock: {juego_actual_db['cantidad'] - cantidad_vendida}")
            
            confirm = messagebox.askyesno("Confirmar Venta", confirm_msg, parent=top)
            if not confirm:
                return

            success, message = self.manager.realizar_venta_gui(
                juego_actual_db['id'],
                id_cliente_seleccionado,
                cantidad_vendida
            )

            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame()
            else:
                messagebox.showerror("Error en Venta", message, parent=top)
                # Podría recargar la ventana si el error es recuperable o de stock
                # Por ahora, si falla, el usuario debe reintentar la operación.

        venta_button = ttk.Button(action_frame, text="Confirmar Venta", command=submit_venta, state=venta_button_state)
        venta_button.pack(pady=10)
        
        # Actualizar el estado del spinbox y botón de venta si cambia la selección de juego
        def update_venta_ui_on_juego_select(event=None):
            selected_game_info = self.juego_map_venta.get(selected_juego_str.get())
            if selected_game_info and selected_game_info['stock_actual'] > 0:
                cantidad_spinbox.config(to=selected_game_info['stock_actual'], state="readonly") # Max es el stock actual
                # Asegurarse que el valor actual del spinbox no exceda el nuevo 'to'
                if self.cantidad_a_vender_var.get() > selected_game_info['stock_actual']:
                    self.cantidad_a_vender_var.set(selected_game_info['stock_actual'])
                if self.cantidad_a_vender_var.get() < 1 and selected_game_info['stock_actual'] > 0:
                     self.cantidad_a_vender_var.set(1) # Mínimo 1 si hay stock
                elif selected_game_info['stock_actual'] == 0: # Aunque no debería estar en la lista
                     self.cantidad_a_vender_var.set(0)
                
                venta_button.config(state=tk.NORMAL)
            else: # No hay juego seleccionado o no tiene stock (no debería pasar con el filtro)
                cantidad_spinbox.config(to=1, state=tk.DISABLED)
                self.cantidad_a_vender_var.set(1)
                venta_button.config(state=tk.DISABLED)

        juego_combo.bind("<<ComboboxSelected>>", update_venta_ui_on_juego_select)
        if juego_options: # Llamar una vez para inicializar el spinbox
            update_venta_ui_on_juego_select()


        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def open_informes_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Informes de Ventas")
        top.geometry("850x600") # Ajustado

        informe_buttons_frame = ttk.Frame(top, padding="10")
        informe_buttons_frame.pack(fill=tk.X)

        informe_display_frame = ttk.Frame(top, padding="10")
        informe_display_frame.pack(expand=True, fill=tk.BOTH)

        self.report_container = ttk.Frame(informe_display_frame) # Usar self para que clear_report_container funcione
        self.report_container.pack(expand=True, fill=tk.BOTH)


        def show_all_sales_report():
            self.clear_report_container() # Correcto
            sales_data = self.manager.get_all_sales_report()
            if sales_data is None:
                ttk.Label(self.report_container, text="Error al obtener el informe de todas las ventas.", foreground="red").pack()
                return
            if not sales_data:
                ttk.Label(self.report_container, text="No hay ventas registradas.").pack()
                return

            cols = ('ID Venta', 'Cliente', 'Videojuego Vendido', 'Precio Unit.', 'Cant. Vendida', 'Total Venta', 'Fecha')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col_name in cols:
                tree.heading(col_name, text=col_name)
                tree.column(col_name, width=110, anchor=tk.W, minwidth=60)
            
            tree.column('ID Venta', width=60, anchor=tk.CENTER, stretch=tk.NO)
            tree.column('Precio Unit.', anchor=tk.E, width=90)
            tree.column('Cant. Vendida', anchor=tk.CENTER, width=90)
            tree.column('Total Venta', anchor=tk.E, width=90)
            tree.column('Fecha', width=140)


            for venta in sales_data:
                total_venta_calculado = venta['precio_venta'] * venta.get('cantidad_vendida', 1)
                tree.insert("", tk.END, values=(
                    venta['id'],
                    venta.get('nombre_cliente', 'N/A'),
                    venta['nombre_videojuego_vendido'],
                    f"${venta['precio_venta']:.2f}",
                    venta.get('cantidad_vendida', 1),
                    f"${total_venta_calculado:.2f}",
                    venta['fecha_venta'].strftime('%Y-%m-%d %H:%M:%S') if venta.get('fecha_venta') else 'N/A'
                ))

            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(self.report_container, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)
            hsb.pack(side=tk.BOTTOM, fill=tk.X)


        def show_total_revenue():
            self.clear_report_container() # Correcto
            total, message = self.manager.get_total_revenue_report()
            if total is not None:
                ttk.Label(self.report_container, text=f"Ingresos Totales: ${total:.2f}", font=("Arial", 16, "bold")).pack(pady=20)
            else:
                ttk.Label(self.report_container, text=message, foreground="red", font=("Arial", 14)).pack(pady=20)


        def show_sales_by_client():
            self.clear_report_container() # Correcto
            data, message = self.manager.get_sales_by_client_report()

            if data is None:
                ttk.Label(self.report_container, text=message, foreground="red").pack()
                return
            if not data:
                ttk.Label(self.report_container, text="No hay ventas agrupadas por cliente para mostrar.").pack()
                return

            cols = ('Cliente', 'Nº Transacciones', 'Total Juegos Comprados', 'Total Gastado')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col_name in cols:
                tree.heading(col_name, text=col_name)
                tree.column(col_name, width=180, anchor=tk.W, minwidth=100)
            
            tree.column('Total Gastado', anchor=tk.E, width=120)
            tree.column('Nº Transacciones', anchor=tk.CENTER, width=120)
            tree.column('Total Juegos Comprados', anchor=tk.CENTER, width=150)


            for row in data:
                tree.insert("", tk.END, values=(
                    row['nombre_cliente'],
                    row['numero_transacciones'],
                    row['total_juegos_comprados'],
                    f"${row['total_gastado']:.2f}"
                ))

            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(self.report_container, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)
            hsb.pack(side=tk.BOTTOM, fill=tk.X)


        ttk.Button(informe_buttons_frame, text="Todas las Ventas", command=show_all_sales_report).pack(side=tk.LEFT, padx=5, pady=5)
        ttk.Button(informe_buttons_frame, text="Ingresos Totales", command=show_total_revenue).pack(side=tk.LEFT, padx=5, pady=5)
        ttk.Button(informe_buttons_frame, text="Ventas por Cliente", command=show_sales_by_client).pack(side=tk.LEFT, padx=5, pady=5)

        # Mostrar "Todas las ventas" por defecto al abrir la ventana de informes
        show_all_sales_report()

        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def clear_report_container(self): # Este método debe estar correctamente definido
        for widget in self.report_container.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = GameStoreApp(root)

    def on_closing():
        if messagebox.askokcancel("Salir", "¿Seguro que quieres salir?"):
            if app.manager:
                app.manager.cerrar_conexion()
            root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_closing)
    root.mainloop()

GameStoreManager: Conectado a la base de datos.
GameStoreManager: Tabla 'videojuegos' verificada/creada.
GameStoreManager: Tabla 'clientes' verificada/creada.
GameStoreManager: Tabla 'ventas' verificada/creada.
GameStoreManager: Conexión cerrada.
